Factors that influence the sharpe ratio:

  * Higher risk free rate reduces the sharpe ratio
  * Higher returns increase the sharpe ratio
  * Higher volatility reduces sharpe ratio

In [39]:
import os
import sys
sys.path.append(os.getcwd())

import os
import pandas as pd
from mom_trans.classical_strategies import (
    calc_returns,
    calc_daily_vol
)


In [37]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df_asset = pd.read_parquet(f'./data/sectors/QQQ_daily.parquet')
df_asset["daily_returns"] = calc_returns(df_asset["close"])
df_asset["daily_vol"] = calc_daily_vol(df_asset["daily_returns"])
df_asset.head()

,open,high,low,close,adj close,volume,symbol,daily_returns,daily_vol
Date,,,,,,,,,
2009-11-23,43.889999,44.400002,43.860001,44.139999,38.552536,91621500,QQQ,NaN,0.011259
2009-11-24,44.119999,44.150002,43.730000,43.990002,38.421520,60670000,QQQ,-0.003398,0.011259
2009-11-25,44.139999,44.220001,44.009998,44.180000,38.587463,37630100,QQQ,0.004319,0.011259
2009-11-27,42.970001,43.830002,42.900002,43.509998,38.002274,61230000,QQQ,-0.015165,0.011259
2009-11-30,43.410000,43.619999,43.110001,43.560001,38.045952,84819400,QQQ,0.001149,0.011259


In [41]:
def calculate_sharpe_ratio(returns, risk_free_rate=0.01, periods_per_year=252):
    """
    Calculate the Sharpe Ratio for a given time series of prices.
    - The script assumes daily data by default (periods_per_year=252)
    - Adjust periods_per_year based on your data frequency (252 for daily, 52 for weekly, 12 for monthly)
    - The risk-free rate is assumed to be 1% by default

    The Sharpe Ratio interpretation:
    - A ratio > 1 is considered good
    - A ratio > 2 is considered very good
    - A ratio > 3 is considered excellent
    - A negative ratio indicates performance worse than the risk-free rate
    
    Parameters:
    prices (array-like): Time series of asset prices
    risk_free_rate (float): Annual risk-free rate (default is 1%)
    periods_per_year (int): Number of periods in a year (252 for daily data, 52 for weekly, 12 for monthly)
    
    Returns:
    float: Sharpe Ratio
    """
    
    # Convert prices to returns
    #returns = pd.Series(prices).pct_change().dropna()
    
    # Calculate excess returns (return - risk_free_rate). To do this
    # first calculate the daily risk free rate.
    rf_per_period = (1 + risk_free_rate)**(1/periods_per_year) - 1
    excess_returns = returns - rf_per_period
    
    # Calculate annualized mean of excess returns
    mean_excess_return = excess_returns.mean() * periods_per_year
    
    # Calculate annualized standard deviation of returns
    annualised_vol = returns.std() * np.sqrt(periods_per_year)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = mean_excess_return / annualised_vol
    
    return sharpe_ratio


In [36]:
df_asset["long_position"] = 1
df_asset['long_return'] = df_asset["daily_returns"].shift(-1) * df_asset["long_position"]

df_asset["perfect_position"] = df_asset["daily_returns"].shift(-1).apply(lambda daily_return: 1 if daily_return > 0 else -1)
df_asset['perfect_return'] = df_asset["daily_returns"].shift(-1) * df_asset["perfect_position"]

long_total_return = df_asset['long_return'].sum()
perfect_total_return = df_asset['perfect_return'].sum()

print(f"Long total return: {long_total_return}")
print(f"Long Sharpe Ratio: {calculate_sharpe_ratio(df_asset['long_return'])}")

print(f"Perfect total return: {perfect_total_return}")
print(f"Perfect Sharpe Ratio: {calculate_sharpe_ratio(df_asset['perfect_return'])}")

Long total return: 2.7341734487044262
Long Sharpe Ratio: 0.8459202884109455
Perfect total return: 33.831454309828146
Perfect Sharpe Ratio: 15.330940347301725
